# Numerical exercises 10

Before running this notebook you need to...

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.style as style 
style.use('tableau-colorblind10')

In the following exercises we study the TSP again, at first using a _Simulated annealing_ algorithm, and then using parallelized genetic algorithm, and in both cases we will compare the results with the ones of the previous lecture.

# Exercise 10.1
## Brief overview of Simulated Annealing

The Simulated Annealing (SA) algorithm comes from an analogy with thermodynamics: when a system is **slowly** (in order to avoid metastable states) cooled down, it reaches the energy minimum.

The main result of this analogy is to consider the loss function $L_{1}(x)$ as the "energy" of the system, so that we define the following Boltzmann like probability distribution:

$$
p(x)= \frac{e^{-\beta L_{1}(x)}}{Z}, \\
$$

So, by increasing $\beta$ we will slowly decrease the "temperature" of the system, and at each MC step we will sample the following distribution with a Metropolis algorithm:

$$
P= \begin{cases}
      e^{-\beta (L_{1}(\vec x') -L_{1}(\vec x))}, & \text{if } L_{1}(\vec x') > L_{1}(\vec x)  \\
      1, & \text{otherwise,}
    \end{cases}
$$

where each of $N$ total cycles will start from the previous final configuration.

# Exercise 10.2
As previously mentioned, in this exercise we will use the [MPI library](https://www.mpi-forum.org/) to parallelize our genetic algorithm. 

Our model falls into the **Single Program Multiple Data (SPMD)** model, where the same executable is used to process different sets of data.